In [ ]:
from loica import *
import matplotlib.pyplot as plt
import getpass

In [ ]:
from flapjack import *
#fj = Flapjack(url_base='flapjack.rudge-lab.org:8000')
fj = Flapjack(url_base='localhost:8000')
fj.log_in(username=input('Flapjack username: '), password=getpass.getpass('Password: '))

In [ ]:
study = fj.create('study', name='Loica testing', description='Test study for demonstrating Loica')

In [ ]:
sfp = fj.create('signal', name='SFP', color='green', description='Simulated fluorescent protein')

In [ ]:
dna = fj.create('dna', name='band_detect')
vector = fj.create('vector', name='band_detect', dnas=dna.id)

In [ ]:
network = GeneticNetwork(vector=vector.id[0])

regs = [Regulator(f'Rep{i}', degradation_rate=1) for i in range(3)]
ahl = Supplement(name='AHL1')
reporter = Reporter(name='SFP', color='green', signal_id=sfp.id[0])

rec1 = Receiver(ahl, regs[0], alpha=[0,10], K=1, n=2)
rec2 = Receiver(ahl, regs[1], alpha=[0,50], K=1, n=2)
not_ = Hill1(regs[1], regs[2], alpha=[50,0], K=1, n=2)
nor = Hill2([regs[0],regs[2]], reporter, alpha=[1,0,0,0], K=[1,1], n=[2,2])

network.add_regulators(regs)
network.add_operators([rec1,rec2,not_,nor])
network.add_reporter(reporter)

In [ ]:
plt.figure(figsize=(3,3), dpi=150)
network.draw()

In [ ]:
def growth_rate(t):
    return gompertz_growth_rate(t, 0.05, 1, 1, 1)

def biomass(t):
    return gompertz(t, 0.05, 1, 1, 1)
    
metab = SimulatedMetabolism(biomass, growth_rate)

media = fj.create('media', name='loica', description='Simulated loica media')
strain = fj.create('strain', name='loica', description='Loica test strain')

In [ ]:
# Create list of samples    
samples = []
concs = np.append(0, np.logspace(-2, 2, 24))
for conc in concs:
    for _ in range(1):
        sample = Sample(genetic_network=network, 
                    metabolism=metab,
                    media=media.id[0],
                    strain=strain.id[0])
        # Add AHL to samples at given concentration
        sample.add_supplement(ahl, conc)
        samples.append(sample)

In [ ]:
biomass_signal = fj.create('signal', name='SOD', description='Simulated OD', color='black')
assay = Assay(samples, 
              n_measurements=100, 
              interval=0.24,
              name='Loica band detect',
              description='Simulated band detect generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

In [ ]:
assay.upload(fj, study.id[0])

In [ ]:
ahl1_id = fj.get('chemical', name='AHL1').id[0]
fig = fj.plot(study=study.id,                     
                vector=vector.id,
                signal=sfp.id,
                type='Induction Curve',
                analyte=ahl1_id,
                function='Mean Expression',
                biomass_signal=biomass_signal.id[0],
                normalize='None',
                subplots='Signal',
                markers='Vector',
                plot='All data points'
       )
fig